In [46]:
import os
from dotenv import load_dotenv
import PyPDF2
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import HumanMessage, AIMessage
from langchain_core.messages import SystemMessage
from langchain.utilities import SerpAPIWrapper
from langchain.agents import load_tools, initialize_agent, AgentType

In [47]:
load_dotenv()
API_KEY = os.getenv("API_KEY")
SER_API_KEY = os.getenv("SER_API_KEY")

In [48]:
class ChatBot:
    def __init__(self, api_key=API_KEY, serp_api_key=SER_API_KEY):
        self.api_key = api_key
        self.serp_api_key = serp_api_key
        self.llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001", google_api_key=self.api_key)
        self.tools = load_tools(["serpapi"], serpapi_api_key=self.serp_api_key)
        self.agent = initialize_agent(self.tools, self.llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
        self.chat_history = [
            SystemMessage(content="You are a helpful assistant.")
        ]
        self.pdf_text = ""

    def normal_chat(self, user_input):
        self.chat_history.append(HumanMessage(content=user_input))
        response = self.llm.invoke(self.chat_history) 
        self.chat_history.append(AIMessage(content=response.content))
        return response.content
     
    def extract_text_from_pdf(self, pdf_path):
        text = ""
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()
        self.pdf_text = text  
        return "PDF text extracted successfully! You can now ask questions based on this PDF."

    
    def upload_pdf(self, pdf_path):
        if os.path.exists(pdf_path):
            return self.extract_text_from_pdf(pdf_path)
        else:
            return "PDF file not found. Please check the file path and try again."

    def answer_from_pdf(self, user_input):
        self.chat_history.append(HumanMessage(content=f"Answer based on the PDF: {self.pdf_text}"))
        self.chat_history.append(HumanMessage(content=user_input))
        response = self.llm.invoke(self.chat_history)
        self.chat_history.append(AIMessage(content=response.content))
        return response.content

    def web_search(self, user_input):
        self.chat_history.append(HumanMessage(content=user_input))
        return self.agent.run(user_input)
    
    def chat(self, user_input):
        if "websearch" in user_input.lower():
            return self.web_search(user_input)
        elif self.pdf_text:
            return self.answer_from_pdf(user_input)
        else:
            return self.normal_chat(user_input)
   

In [49]:
chatbot = ChatBot()

In [53]:
response = chatbot.web_search("websearch can you tell me what is today's date?")
print(response)



> Entering new AgentExecutor chain...
I need to find out the current date.

Action: Search
Action Input: "current date"
Observation: Thursday, April 10, 2025
Thought:I now know the final answer
Final Answer: Today is Thursday, April 10, 2025.

> Finished chain.
Today is Thursday, April 10, 2025.


In [50]:
response = chatbot.chat("what is the height of burj khalifa")
print(response)

The Burj Khalifa is 828 meters (2,717 feet) tall.


In [51]:
chatbot.upload_pdf('The Great Gatsby.pdf')

'PDF text extracted successfully! You can now ask questions based on this PDF.'

In [52]:
res = chatbot.chat("What is writer of this novelt?")
print(res)

F. Scott Fitzgerald is the writer of the novel.


In [45]:
response = chatbot.normal_chat("what is the height of burj khalifa")
print(response)

The Burj Khalifa is 828 meters (2,717 feet) tall.
